# Setting

In [1]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [2]:
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


# 前処理

In [4]:
#Enumerate Homeplanet
train = df_train.copy()

title_mapping = {"Europa": 1, "Earth": 2, "Mars": 3}
train['HomePlanet'] = train['HomePlanet'].map(title_mapping)
train['HomePlanet'] = train['HomePlanet'].fillna(0)

train[['Cabin_1', 'Cabin_2', 'Cabin_3']] = train['Cabin'].str.split('/', expand=True)
train.drop('Cabin', axis=1, inplace=True)

cabin_1_mapping = {"B": 1, "F": 2, "A": 3, "G": 4, "E": 5, "D": 6, "C": 7, "T": 8}
train['Cabin_1'] = train['Cabin_1'].map(cabin_1_mapping)
train['Cabin_1'] = train['Cabin_1'].fillna(0)

cabin_3_mapping = {"P": 1, "S": 2}
train['Cabin_3'] = train['Cabin_3'].map(cabin_3_mapping)
train['Cabin_3'] = train['Cabin_3'].fillna(0)

train['Cabin_2'] = train['Cabin_2'].fillna(9999)
train['Cabin_2'] = train['Cabin_2'].astype(int) + 1
train['Cabin_2'] = train['Cabin_2'].replace(10000, 0)


train['Fare'] = train[['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
cols = ['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']
for col in cols:
    train[col] = train[col].fillna(train[col].median())


train.loc[train["Transported"] == 1, "Age"] = train[train["Transported"] == 1]["Age"].fillna(train[train["Transported"] == 1]["Age"].mean())
train.loc[train["Transported"] == 0, "Age"] = train[train["Transported"] == 0]["Age"].fillna(train[train["Transported"] == 0]["Age"].mean())

print(train["Age"].mean())

train.drop('Destination', axis=1, inplace=True)

vip_mapping = {False: 1, True: 2}
train['VIP'] = train['VIP'].map(vip_mapping)
train['VIP'] = train['VIP'].fillna(0)

cryo_mapping = {False: 1, True: 2}
train['CryoSleep'] = train['CryoSleep'].map(cryo_mapping)
train['CryoSleep'] = train['CryoSleep'].fillna(0)

# 学習用の目的変数を保存
y = train['Transported']

train.drop(['Name', 'Transported', 'PassengerId'], axis = 1, inplace = True)

28.827968418500845


In [5]:
#Cleaning Tes
test = df_test.copy()
title_mapping = {"Europa": 1, "Earth": 2, "Mars": 3}
test['HomePlanet'] = test['HomePlanet'].map(title_mapping)
test['HomePlanet'] = test['HomePlanet'].fillna(0)

test[['Cabin_1', 'Cabin_2', 'Cabin_3']] = test['Cabin'].str.split('/', expand=True)
test.drop('Cabin', axis=1, inplace=True)

cabin_1_mapping = {"B": 1, "F": 2, "A": 3, "G": 4, "E": 5, "D": 6, "C": 7, "T": 8}
test['Cabin_1'] = test['Cabin_1'].map(cabin_1_mapping)
test['Cabin_1'] = test['Cabin_1'].fillna(0)

cabin_3_mapping = {"P": 1, "S": 2}
test['Cabin_3'] = test['Cabin_3'].map(cabin_3_mapping)
test['Cabin_3'] = test['Cabin_3'].fillna(0)

test['Cabin_2'] = test['Cabin_2'].fillna(9999)
test['Cabin_2'] = test['Cabin_2'].astype(int) + 1
test['Cabin_2'] = test['Cabin_2'].replace(10000, 0)


test['Fare'] = test[['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
cols = ['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']
for col in cols:
    test[col] = test[col].fillna(test[col].median())

test['Age'] = test['Age'].fillna(28.827968418500845)

test.drop('Destination', axis=1, inplace=True)

vip_mapping = {False: 1, True: 2}
test['VIP'] = test['VIP'].map(vip_mapping)
test['VIP'] = test['VIP'].fillna(0)

cryo_mapping = {False: 1, True: 2}
test['CryoSleep'] = test['CryoSleep'].map(cryo_mapping)
test['CryoSleep'] = test['CryoSleep'].fillna(0)




test.drop(['Name', 'PassengerId'], axis = 1, inplace = True)

In [6]:
data = df_train.copy()

# Step 1: Check for missing values
missing_values = data.isnull().sum()

# Step 2: Data type conversion and initial transformations
# Convert CryoSleep and VIP to boolean
data['CryoSleep'] = data['CryoSleep'].map({'True': True, 'False': False})
data['VIP'] = data['VIP'].map({'True': True, 'False': False})

# Step 3: Feature engineering
# Split Cabin into deck, num, and side
data[['Deck', 'Num', 'Side']] = data['Cabin'].str.split('/', expand=True)

# Split PassengerId into group and number
data[['Group', 'GroupNum']] = data['PassengerId'].str.split('_', expand=True)

# Create a new feature: Total expenditure
data['TotalExpenditure'] = data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)

# Step 4: Check target distribution
target_distribution = data['Transported'].value_counts(normalize=True)

# Display the processed dataset overview and results
processed_overview = {
    "Missing Values": missing_values,
    "Target Distribution": target_distribution,
    "Columns After Feature Engineering": data.columns.tolist()
}

In [7]:
# trainとtestを保存する
train.to_csv('train_v2.csv', index=False)
test.to_csv('test_v2.csv', index=False)

In [8]:
train.head()

,HomePlanet,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_1,Cabin_2,Cabin_3,Fare
0,1.0,1.0,39.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1.0,0.0
1,2.0,1.0,24.0,1.0,109.0,9.0,25.0,549.0,44.0,2.0,1,2.0,736.0
2,1.0,1.0,58.0,2.0,43.0,3576.0,0.0,6715.0,49.0,3.0,1,2.0,10383.0
3,1.0,1.0,33.0,1.0,0.0,1283.0,371.0,3329.0,193.0,3.0,1,2.0,5176.0
4,2.0,1.0,16.0,1.0,303.0,70.0,151.0,565.0,2.0,2.0,2,2.0,1091.0


## モデル構築〜予測

In [9]:
!pip install catboost
!pip install lightgbm
!pip install xgboost
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.8 MB/s eta 0:00:00


In [10]:
import catboost
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [11]:
cat = catboost.CatBoostClassifier()
lgbm = lgb.LGBMClassifier()
xgb = xgb.XGBClassifier()
rfc = RandomForestClassifier()
lr = LogisticRegression()
knn = KNeighborsClassifier()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)
y_train = y_train.astype(int)

# 形状確認
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6085, 13)
(2608, 13)
(6085,)
(2608,)


In [13]:
# 学習
cat.fit(X_train, y_train)
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)
rfc.fit(X_train, y_train)
lr.fit(X_train, y_train)
knn.fit(X_train, y_train)

Learning rate set to 0.022275
0:	learn: 0.6824546	total: 52.1ms	remaining: 52.1s
1:	learn: 0.6724307	total: 56.5ms	remaining: 28.2s
2:	learn: 0.6620011	total: 60.3ms	remaining: 20s
3:	learn: 0.6527150	total: 65.9ms	remaining: 16.4s
4:	learn: 0.6434609	total: 70.1ms	remaining: 13.9s
5:	learn: 0.6355171	total: 77.5ms	remaining: 12.8s
6:	learn: 0.6282894	total: 81.8ms	remaining: 11.6s
7:	learn: 0.6217603	total: 85.7ms	remaining: 10.6s
8:	learn: 0.6145716	total: 90.5ms	remaining: 9.97s
9:	learn: 0.6066957	total: 96.5ms	remaining: 9.55s
10:	learn: 0.6007055	total: 104ms	remaining: 9.32s
11:	learn: 0.5939195	total: 117ms	remaining: 9.67s
12:	learn: 0.5863886	total: 123ms	remaining: 9.36s
13:	learn: 0.5814849	total: 127ms	remaining: 8.96s
14:	learn: 0.5761421	total: 133ms	remaining: 8.75s
15:	learn: 0.5714695	total: 138ms	remaining: 8.5s
16:	learn: 0.5654098	total: 144ms	remaining: 8.3s
17:	learn: 0.5602264	total: 148ms	remaining: 8.07s
18:	learn: 0.5553207	total: 155ms	remaining: 8.01s
19:	l

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KNeighborsClassifier()

In [14]:
# 予測
cat_pred = cat.predict(X_test)
lgbm_pred = lgbm.predict(X_test)
xgb_pred = xgb.predict(X_test)
rfc_pred = rfc.predict(X_test)
lr_pred = lr.predict(X_test)
knn_pred = knn.predict(X_test)

In [15]:
# それぞれのモデルのaccuracy_scokreをリストにまとめる
models = [rfc, lr, knn, cat, lgbm, xgb]
scores = []
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# scoresを降順にソート
sorted_scores = sorted(scores, reverse=True)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Learning rate set to 0.022275
0:	learn: 0.6824546	total: 22.4ms	remaining: 22.4s
1:	learn: 0.6724307	total: 46.1ms	remaining: 23s
2:	learn: 0.6620011	total: 68.2ms	remaining: 22.7s
3:	learn: 0.6527150	total: 87.6ms	remaining: 21.8s
4:	learn: 0.6434609	total: 98.5ms	remaining: 19.6s
5:	learn: 0.6355171	total: 112ms	remaining: 18.6s
6:	learn: 0.6282894	total: 125ms	remaining: 17.8s
7:	learn: 0.6217603	total: 131ms	remaining: 16.2s
8:	learn: 0.6145716	total: 153ms	remaining: 16.8s
9:	learn: 0.6066957	total: 168ms	remaining: 16.6s
10:	learn: 0.6007055	total: 181ms	remaining: 16.3s
11:	learn: 0.5939195	total: 201ms	remaining: 16.5s
12:	learn: 0.5863886	total: 212ms	remaining: 16.1s
13:	learn: 0.5814849	total: 227ms	remaining: 16s
14:	learn: 0.5761421	total: 241ms	remaining: 15.8s
15:	learn: 0.5714695	total: 261ms	remaining: 16.1s
16:	learn: 0.5654098	total: 278ms	remaining: 16.1s
17:	learn: 0.5602264	total: 287ms	remaining: 15.7s
18:	learn: 0.5553207	total: 293ms	remaining: 15.1s
19:	learn:

In [16]:
# scoresをモデル名とともに表示

# モデル名とスコアの対応付けを辞書で作成
model_names = ['RandomForestClassifier', 'LogisticRegression', 'KNN', 'CatBoost', 'lightGBM', 'XGBoost',]
model_scores = dict(zip(model_names, scores))

# スコアを降順にソートし、モデル名とともに表示
for model_name, score in sorted(model_scores.items(), key=lambda item: item[1], reverse=True):
    print(f'{model_name}: {score}')

lightGBM: 0.8117331288343558
CatBoost: 0.8048312883435583
XGBoost: 0.7998466257668712
RandomForestClassifier: 0.7887269938650306
LogisticRegression: 0.7730061349693251
KNN: 0.754601226993865


In [29]:
# テストデータで予測
test_pred = lgbm.predict(test)

In [30]:
# test_predで0の場合はFalse、1の場合はTrueに変換
test_pred = test_pred.astype(bool)

In [31]:
test_pred.shape

(4277,)

## 提出データ作成

In [33]:
sample = pd.read_csv('sample_submission.csv')
# sampleデータのTransported列にtest_predを代入
sample['Transported'] = test_pred

In [35]:
# データ出力
sample.to_csv('submission.csv', index=False)